# Useful links

- [Choosing colourmaps for plotting gridded data](https://matplotlib.org/stable/users/explain/colors/colormaps.html) - you can reverse a colourmap by adding '_r' to its name, eg. 'RdBu_r'
- [List of named colours for plotting time series](https://matplotlib.org/stable/gallery/color/named_colors.html)

# Import required packages

In [ ]:
import xarray as xr                                                  # handles gridded data
import numpy as np                                                   # numerical functions etc
import pandas as pd                                                  # handles tables of data (.csv, spreadsheets etc)
import re                                                            # allows manipulation of text strings

import matplotlib                                                    # plotting functions generally
import matplotlib.pyplot as plt                                      # specific plotting function we'll use a lot, so we'll give it a shorter name
from matplotlib.patches import Rectangle                             # specific plotting function with an annoyingly long name

import statsmodels.nonparametric.smoothers_lowess as sm_lowess       # nonparametric smoother

import cartopy                                                       # plotting geographical features
import regionmask                                                    # convert shapefile to binary 0,1 mask
import geopandas as gpd                                              # shapefiles

from geopy.geocoders import Nominatim                                # get lat & lon coordinates from a place name
from dateutil.relativedelta import relativedelta                     # calculate differences in times (needed to decode weird timestamps)

# filter out any annoying user warnings
import warnings
warnings.filterwarnings("ignore", category = UserWarning)

# you might get an error like 'ERROR 1: PROJ: proj_create_from_database: Open of /rds/general/user/cb2714/home/anaconda3/envs/wwa/share/proj failed' - you can ignore this!

# Useful functions

In [ ]:
def wrap_lon(ds):
    
    # 'wrap' longitudes from (0,360) to (-180,180) & sort into ascending order
    
    if "longitude" in ds.coords:
        lon = "longitude"
        lat = "latitude"
    elif "lon" in ds.coords:
        lon = "lon"
        lat = "lat"
    else: 
        # can only wrap longitude
        return ds
    
    if ds[lon].max() > 180:
        ds[lon] = (ds[lon].dims, (((ds[lon].values + 180) % 360) - 180), ds[lon].attrs)
        
    if lon in ds.dims:
        ds = ds.reindex({ lon : np.sort(ds[lon]) })
        ds = ds.reindex({ lat : np.sort(ds[lat]) })
    return ds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def decode_times(ts):
    
    # manually decode times
    
    inc = re.sub(" .+", "", ts.time.units)
    startdate = pd.Timestamp(re.sub(".+since ", "", ts.time.units)+' 00:00:00.000000').to_pydatetime()
    
    if inc == "years":
        new_times = [np.datetime64(startdate + relativedelta(years = i)) for i in ts.time.values]
    elif inc == "months":
        new_times = [np.datetime64(startdate + relativedelta(months = i)) for i in ts.time.values]
    elif inc == "days":
        new_times = [np.datetime64(startdate + relativedelta(days = i)) for i in ts.time.values]
    elif inc == "hours":
        new_times = [np.datetime64(startdate + relativedelta(hours = i)) for i in ts.time.values]
    else:
        print("TBD: " +inc)
        return
        
    ts = ts.assign_coords(time = new_times)
    
    return ts

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def get_latlon(city):
    
    # retrieve lat & lon for given place name
    location = Nominatim(user_agent="GetLoc").geocode(city)
    if location is None:
        return {"lon" : None, "lat" : None}
    else:
        return {"lon" : location.longitude, "lat" : location.latitude}
        
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def doy_axes(ax = None):

    # make nicer axis labels when you're plotting against day of year
    if ax is None: ax = plt.gca()
    
    # define axis labels for DOY plotting
    days = pd.date_range(start = "2020-01-01", end = "2020-12-31")
    lineticks = [i+1 for i in range(366) if (days[i].day) == 1]
    labelticks = [i+1 for i in range(366) if (days[i].day) == 16]
    labels = [days[i].strftime("%b") for i in range(366) if (days[i].day) == 16]

    # actually draw the axes & gridlines
    ax.set_xticks(labelticks)
    ax.set_xticklabels(labels)
    [ax.axvline(x, color = "k", alpha = 0.1, lw = 1) for x in lineticks]
    ax.set_xlabel("")

# Load & plot gridded data

## Load a netcdf (.nc) file

In [ ]:
# open a dataset and clean up the lats & lons
ds = wrap_lon(xr.open_dataset("era5_tp_daily_eu_extended_10.7-25E_46-52.3N_1950-2024_su.nc"))    # change filename to use different data

# sometimes you'll try to load data with a date format that python can't decode (ie. annual data from the climate explorer) - in that case, try
# ds = decode_times(xr.open_dataset(""era5_tp_daily_eu_extended_10.7-25E_46-52.3N_1950-2024_su.nc", decode_times = False)).tp

# select the variable you're interested in (even though there may only be one in the dataset)
ds = ds.tp

## Select only land cells

If your chosen area contains both land and sea, you usually only want to look at values over the land.  
The `regionmask` package provides predefined shapefiles for coastlines, which can be used to select only the land surface.

In [ ]:
# create a mask where all points in the sea are NA
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_50.mask(ds.lon, ds.lat)

# mask out any values that aren't on land
ds = ds.where(rm == 0)

## Plot a single day

In [ ]:
# select a single day of data to plot
ds_event = ds.sel(time = "2024-09-14")

# get the max & min lat & lon so we can fix the plotting area
data_area = (ds.lon.min(), ds.lon.max(), ds.lat.min(), ds.lat.max())

# MAKE THE {PLOT}
fig = ds_event.plot(cmap = "YlGnBu",                                                       # colour scheme to use for plotting
                    vmin = 0, vmax = 100,                                                  # set min & max values for colour scale
                    subplot_kws = {"projection" : cartopy.crs.PlateCarree(),               # tell it which projection to use
                                   "extent" : data_area                                    # only plot the area covered by the data
                                  },     
                    cbar_kwargs = {"location" : "bottom",                                  # move the colourbar (scale bar) to the bottom
                                   "label" : "Precipitation (mm/day)"})                    # relabel the colourbar


# Add some extra labels to the plot
ax = fig.axes                                                                          # give a name to the figure axes so that we can add extra elements
ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.6)                            # add national borders
ax.gridlines(color = "k", alpha = 0.1, draw_labels = ["bottom", "left"])                   # add gridlines
ax.coastlines(color = "k")                                                                 # add coastlines

# save the figure for later use
plt.savefig("example-plot.png", bbox_inches = "tight")                                     # bbox_inches removes any white space around the edges of the plot

## Plot several days together

In [ ]:
# select a range of dates to plot (not too many or the plots will be tiny!)
ds_event = ds.sel(time = slice("2024-09-12", "2024-09-16"))

# get the extent of the data (this will help to avoid messy plots when we add geographic features)
data_area = (ds.lon.min(), ds.lon.max(), ds.lat.min(), ds.lat.max())

fig = ds_event.plot(col = "time",                                       # each panel should show a different time
                    col_wrap = 3,                                       # number of panels to show per row
                    cmap = "YlGnBu",                                    # colour scheme to use for plotting
                    vmin = 0, vmax = 100,                               # set min & max values for colour scale
                    # specify the projection for each panel
                    subplot_kws = {
                        "projection" : cartopy.crs.PlateCarree(),       # map projection so that we can add geographic features
                        "extent" : data_area                            # only plot the area covered by the data
                    },     
                    # specify how the colour bar should look
                    cbar_kwargs = {
                        "location" : "bottom",                          # move the colourbar (scale bar) to the bottom
                        "label" : "Precipitation (mm/day)",             # relabel the colourbar
                        "fraction" : 0.025,                             # how big the colourbar should be
                        "pad" : 0.1                                     # how much space between the colourbar and the panels
                    })            

# this time, we have to add the borders and gridlines to each panel individually
for ax in fig.axs.flat:
    ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.6)                # add national borders
    ax.gridlines(color = "k", alpha = 0.1)                                         # add gridlines but don't draw the labels
    ax.coastlines(color = "k")                                                     # add coastlines
    
for ax in fig.axs[:,0]: ax.gridlines(alpha = 0, draw_labels = ["left"])            # y-labels on leftmost column
for ax in fig.axs[-1,:]: ax.gridlines(alpha = 0, draw_labels = ["bottom"])         # x-labels on the bottom row 

# Optional: manually set the title of each panel (make sure you make as many titles as there are panels!)
# titles = ["Day 1", "Day 2", "Day 3", "Day 4", "Day 5"]
# for ax, title in zip(fig.axs.flat, titles):
#     ax.set_title(title)

## Add sites where impacts occurred

In [ ]:
# make a list of the locations where impacts were reported 
# these could be a village, city, state, or country - just has to be a real place
# you may need to specify the country if more than one place exists with the same name, eg. "Vienna, Austria"
affected_sites = ["Vienna", "Graz", "Glucholazy", "Litovel", "Lower Austria"]

# use user-defined function to get the coordinates and turn into a table
px = pd.DataFrame([get_latlon(s) for s in affected_sites])

In [ ]:
# replicate the plot that we made above
fig = ds_event.plot(col = "time",                                       # each panel should show a different time
                    col_wrap = 3,                                       # number of panels to show per row
                    cmap = "YlGnBu",                                    # colour scheme to use for plotting
                    vmin = 0, vmax = 100,                               # set min & max values for colour scale
                    # specify the projection for each panel
                    subplot_kws = {
                        "projection" : cartopy.crs.PlateCarree(),       # map projection so that we can add geographic features
                        "extent" : data_area                            # only plot the area covered by the data
                    },     
                    # specify how the colour bar should look
                    cbar_kwargs = {
                        "location" : "bottom",                          # move the colourbar (scale bar) to the bottom
                        "label" : "Precipitation (mm/day)",             # relabel the colourbar
                        "fraction" : 0.025,                             # how big the colourbar should be
                        "pad" : 0.1                                     # how much space between the colourbar and the panels
                    })            

# this time, we have to add the borders and gridlines to each panel individually
for ax in fig.axs.flat:
    ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.6)                # add national borders
    ax.gridlines(color = "k", alpha = 0.1)                                         # add gridlines but don't draw the labels
    ax.coastlines(color = "k")                                                     # add coastlines
    ax.scatter(px.lon, px.lat, marker = "x", color = "red")                        # **NEW ELEMENT** add points where impacts were reported
    
for ax in fig.axs[:,0]: ax.gridlines(alpha = 0, draw_labels = ["left"])            # y-labels on leftmost column
for ax in fig.axs[-1,:]: ax.gridlines(alpha = 0, draw_labels = ["bottom"])         # x-labels on the bottom row 

# Optional: manually set the title of each panel (make sure you make as many titles as there are panels!)
# titles = ["Day 1", "Day 2", "Day 3", "Day 4", "Day 5"]
# for ax, title in zip(fig.axs.flat, titles):
#     ax.set_title(title)

# Daily anomalies

## Calculate daily anomalies

In [ ]:
# select only the period that we want to compare (could be the whole year, could be just selected months)
ds_subset = ds.sel(time = ds.time.dt.month.isin([5,6,7,8,9]))

# calculate the mean daily rainfall during those months from 1990-2020 (standard climatology period)
ds_clim = ds_subset.sel(time = slice("1990", "2020")).mean("time")

# calculate the daily anomalies with respect to this climatology
ds_anom = ds_subset - ds_clim

## Plot daily anomalies

In [ ]:
# select just the dates we're interested in plotting
ds_event_anom = ds_anom.sel(time = slice("2024-09-12", "2024-09-16"))

# replicate the plot that we made above, but this time we should use a DIVERGENT colour scheme and set the vmin and vmax to be symmetric about zero
fig = ds_event_anom.plot(col = "time",                                       # each panel should show a different time
                         col_wrap = 3,                                       # number of panels to show per row
                         cmap = "BrBG",                                    # colour scheme to use for plotting
                         vmin = -100, vmax = 100,                               # set min & max values for colour scale
                         # specify the projection for each panel
                         subplot_kws = {
                             "projection" : cartopy.crs.PlateCarree(),       # map projection so that we can add geographic features
                             "extent" : data_area                            # only plot the area covered by the data
                         },     
                         # specify how the colour bar should look
                         cbar_kwargs = {
                             "location" : "bottom",                          # move the colourbar (scale bar) to the bottom
                             "label" : "Precipitation (mm/day)",             # relabel the colourbar
                             "fraction" : 0.025,                             # how big the colourbar should be
                             "pad" : 0.1                                     # how much space between the colourbar and the panels
                         })     

# this time, we have to add the borders and gridlines to each panel individually
for ax in fig.axs.flat:
    ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.6)                # add national borders
    ax.gridlines(color = "k", alpha = 0.1)                                         # add gridlines but don't draw the labels
    ax.coastlines(color = "k")                                                     # add coastlines
    ax.scatter(px.lon, px.lat, marker = "x", color = "red")                        # **NEW ELEMENT** add points where impacts were reported
    
for ax in fig.axs[:,0]: ax.gridlines(alpha = 0, draw_labels = ["left"])            # y-labels on leftmost column
for ax in fig.axs[-1,:]: ax.gridlines(alpha = 0, draw_labels = ["bottom"])         # x-labels on the bottom row 

# Maps of accumulations (the 'event map')

In [ ]:
# how many days to accumulate? (how long did the event last)
ndays = 4

# get the n-day rolling mean (could also use the sum for precipitation, if preferred)
ds_nday = ds.rolling(time = ndays, center = False).mean()

In [ ]:
# select the date on which the event finished
nday_event = ds_nday.sel(time = "2024-09-16")

# plot the n-day event
fig = nday_event.plot(cmap = "YlGnBu",                                    # colour scheme to use for plotting
                      vmin = 0, vmax = 75,                               # set min & max values for colour scale
                      # specify the projection for each panel
                      subplot_kws = {
                          "projection" : cartopy.crs.PlateCarree(),       # map projection so that we can add geographic features
                          "extent" : data_area                            # only plot the area covered by the data
                      },     
                      # specify how the colour bar should look
                      cbar_kwargs = {
                          "location" : "bottom",                          # move the colourbar (scale bar) to the bottom
                          "label" : "Precipitation (mm/day)",             # relabel the colourbar
                          "fraction" : 0.025,                             # how big the colourbar should be
                          "pad" : 0.1                                     # how much space between the colourbar and the panels
                      })            

ax = fig.axes
ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.6)            # add national borders
ax.gridlines(color = "k", alpha = 0.1, draw_labels = ["bottom", "left"])   # add gridlines
ax.coastlines(color = "k")                                                 # add coastlines
ax.scatter(px.lon, px.lat, marker = "x", color = "red")                    # add points where impacts were reported

# give this panel a better title
ax.set_title("Average daily rainfall from September 13th-16th")

# Averaging over an area

## Square/rectangular area


If you're looking at a region with both land and sea, remember to mask out the land first (see below)

In [ ]:
# set the region we want to draw a box around
xn,xx,yn,yx = [14,18,47,51]

In [ ]:
# check the region by adding it to the plot
fig = nday_event.plot(cmap = "YlGnBu",                                    # colour scheme to use for plotting
                      vmin = 0, vmax = 75,                               # set min & max values for colour scale
                      # specify the projection for each panel
                      subplot_kws = {
                          "projection" : cartopy.crs.PlateCarree(),       # map projection so that we can add geographic features
                          "extent" : data_area                            # only plot the area covered by the data
                      },     
                      # specify how the colour bar should look
                      cbar_kwargs = {
                          "location" : "bottom",                          # move the colourbar (scale bar) to the bottom
                          "label" : "Precipitation (mm/day)",             # relabel the colourbar
                          "fraction" : 0.025,                             # how big the colourbar should be
                          "pad" : 0.1                                     # how much space between the colourbar and the panels
                      })            

ax = fig.axes
ax.add_feature(cartopy.feature.BORDERS, ls = "--", alpha = 0.6)            # add national borders
ax.gridlines(color = "k", alpha = 0.1, draw_labels = ["bottom", "left"])   # add gridlines
ax.coastlines(color = "k")                                                 # add coastlines
ax.scatter(px.lon, px.lat, marker = "x", color = "red")                    # add points where impacts were reported

# draw the box on the map
ax.plot([xn,xn,xx,xx,xn], [yn,yx,yx,yn,yn], color = "darkred", lw = 2, alpha = 0.5)

# give this panel a better title
ax.set_title("Average daily rainfall from September 13th-16th")

In [ ]:
# extract the data over this smaller region only
ds_box = ds.sel(lon = slice(xn,xx), lat = slice(yn,yx))

# compute spatial average
ts_box = ds_box.mean(["lat", "lon"])

# save the data we've extracted
ts_box.to_netcdf("timeseries_pr-box.nc")

## Pre-loaded regions

The `regionmask` package also provides predefined shapefiles for countries and US states.  
To use a different regionmask, replace `countries_50` with `us_states_50`

In [ ]:
# load the regionmask so that we can manipulate it
sf = regionmask.defined_regions.natural_earth_v5_0_0.countries_50.to_geodataframe()

# we'll need to filter the shapefile to idenfity the region(s) of interest
# look at the shapefile to see which column we should filter on
sf

In [ ]:
# then we can select a single country, or a list of countries
sf_subset = sf.loc[sf.names.isin(["Austria", "Czechia"])]
sf_subset.plot()

In [ ]:
# we can use this to create a regionmask for the dataset `ds` by selecting only points that lie within the selected region
rm = regionmask.mask_geopandas(sf_subset, ds.lon, ds.lat)

# coloured points will be included in the selected area
rm.plot()

In [ ]:
# Finally, we can select just the region(s) we're interested in, and average over that
ts_countries = ds.where(~np.isnan(rm)).mean(["lat", "lon"])

# save the data we've extracted
ts_countries.to_netcdf("timeseries_pr-countries.nc")

## Shapefiles from other sources

A 'shapefile' is just a collection of points and shapes defining a region, stored as a set of specific files that have to be loaded together.  
When you download a shapefile, put all the files into a single folder and tell Python to open that folder.

Handy sources for shapefiles:
- [The Humanitarian Data Exchange](https://data.humdata.org/dataset?ext_subnational=1&ext_administrative_divisions=1&q=&sort=if(gt(last_modified%2Creview_date)%2Clast_modified%2Creview_date)%20desc&ext_page_size=25) has shapefiles for lots of administrative regions in different countries
- [Hydrobasins](https://www.hydrosheds.org/products/hydrobasins) provides nested river basins - you'll have to play around to figure out which 'layer' you need, I usually start at 4 (scroll right down to the bottom to download per continent and start with levels 1-6, the files are much smaller)
- [Natural Earth](https://www.naturalearthdata.com/features/) has both political and physical shapefiles


In [ ]:
# load the shapefile by loading the directory that contains the files
sf = gpd.read_file("sf_hybas04/")

# if your shapefile covers a larger area than your data, clip the shapefile to show only the same area
data_area = (ds.lon.min().values.item(), ds.lat.min().values.item(), ds.lon.max().values.item(),  ds.lat.max().values.item())
sf = sf.clip(data_area)

# plot the edge of each shape
sf.boundary.plot()

In [ ]:
# we can shade just one of the shapes at a time to decide which one(s) we want to use
fig = sf.boundary.plot()
sf.iloc[[12]].plot(ax = fig.axes)

In [ ]:
# once you've chosen a region, we select just that subset of the data
sf_subset = sf.iloc[[10,12]]

In [ ]:
# create a regionmask as we did before
rm = regionmask.mask_geopandas(sf_subset, ds.lon, ds.lat)

# coloured points will be included in the selected area
rm.plot()

In [ ]:
# Finally, we can select just the region(s) we're interested in, and average over that
ts_sf = ds.where(~np.isnan(rm)).mean(["lat", "lon"])

# save the data we've extracted
ts_sf.to_netcdf("timeseries_pr-shapefile.nc")